# Iterables and Iterators


- Iterable, refers to an object that can be iterated.
- Iterator, refers to an object that implements python `Iterator Protocol`.

## Iterables Definition

In the most basic form, they are something that can be iterated over, however, this does not mean that it has to be based on indexing.
- Sequences are iterables, but so are other objects that are not sequences.
- As long as we have a way of getting a next element out of the container of objects we can call it iterable.

In [ ]:
print("---- INFINITE ITERABLE ----")
class Squares:
  def __init__(self):
    self.i = 0

  def next_(self):
    result = self.i ** 2
    self.i += 1
    return result

# In this basic form we have a Squares object that can return us the next
# square in its index each time we call the method.
sq1 = Squares()
for i in range(5):
  print(f"Square returns {sq1.next_()} on iteration: {i}")

---- INFINITE ITERABLE ----
Square returns 0 on iteration: 0
Square returns 1 on iteration: 1
Square returns 4 on iteration: 2
Square returns 9 on iteration: 3
Square returns 16 on iteration: 4


In the aproach above the object will continue to iterate and the only way to restart it is to instantiate a new one.

A way in which we could handle this is to give a length property to our iterable and raise a `StopIteration` exception.

- We will then use now the `__next__` method python uses to map to `next()`.
- We will have an `StopIteration` exception once we iterate over all elements.

In [ ]:
print("---- FINITE ITERABLE ----")
class Squares:
  def __init__(self, length):
    self.length = length
    self.i = 0
    
  def __len__(self):
    return self.length

  def __next__(self):
    if self.i >= self.length:
      raise StopIteration
    else:
      result = self.i ** 2
      self.i += 1
      return result

sq1 = Squares(length=5)
i = 0
while True:
  try:
    print(f"Square returns {next(sq1)} on iteration: {i}")
    i += 1
  except StopIteration:
    break

---- FINITE ITERABLE ----
Square returns 0 on iteration: 0
Square returns 1 on iteration: 1
Square returns 4 on iteration: 2
Square returns 9 on iteration: 3
Square returns 16 on iteration: 4


In [ ]:
print("---- RANDOM ITERABLE ----")
# to make it not an index based iterable (sequence) let's use random

import random

class RandomNumbers:
  def __init__(self, length, *, range_min=0, range_max=10):
    self.length = length
    self.range_min = range_min
    self.range_max = range_max
    self.num_requested = 0

  def __len__(self):
    return self.length

  def __next__(self):
    if self.num_requested >= self.length:
      raise StopIteration
    else:
      self.num_requested += 1
      number = random.randint(self.range_min, self.range_max)
      return number

rn1 = RandomNumbers(length=5)
i = 0
while True:
  try:
    print(f"RandomNumbers returns {next(rn1)} on iteration: {i}")
    i += 1
  except StopIteration:
    break

---- RANDOM ITERABLE ----
RandomNumbers returns 3 on iteration: 0
RandomNumbers returns 4 on iteration: 1
RandomNumbers returns 2 on iteration: 2
RandomNumbers returns 7 on iteration: 3
RandomNumbers returns 5 on iteration: 4


## Iterator Definition

While an iterable is an object that can be iterated, for us to use the built-in python functions to iterate an object mus implement the **ITERATOR PROTOCOL**.
- the `__next__` method should be created to handle the next element and raise the `StopIteration` exception.
- the `__iter__` method should be created to return the class instance (`self`) // it looks for an iterator so if returns self then self is the iterator.

In [5]:
print("---- ITERATOR ----")
class Squares:
  def __init__(self, length):
    self.length = length
    self.i = 0
    
  def __len__(self):
    return self.length

  def __next__(self):
    print("__next__ called")
    if self.i >= self.length:
      raise StopIteration
    else:
      result = self.i ** 2
      self.i += 1
      return result

  def __iter__(self):
    print("__iter__ called")
    return self

sq1 = Squares(length=5)
print('Now we can loop through our iterator with python built ins')

for i, sq in enumerate(sq1):
  print(f"iteration: {i} returns value: {sq}")

# however, the problem is that our iterator is still consumed once it is ran.
# we would have to instantiate again our class to run it again.

print("we could also use the sorted function")
sq1 = Squares(length=5)
print(f"Squares sorted in reverse is: {sorted(sq1, reverse=True)}")

---- ITERATOR ----
Now we can loop through our iterator with python built ins
__iter__ called
__next__ called
iteration: 0 returns value: 0
__next__ called
iteration: 1 returns value: 1
__next__ called
iteration: 2 returns value: 4
__next__ called
iteration: 3 returns value: 9
__next__ called
iteration: 4 returns value: 16
__next__ called
we could also use the sorted function
__iter__ called
__next__ called
__next__ called
__next__ called
__next__ called
__next__ called
__next__ called
Squares sorted in reverse is: [16, 9, 4, 1, 0]


In [7]:
# what python actualy does is to call first function iter()
sq = Squares(5)
sq_iterator = iter(sq) # it gets the iterator
print(id(sq), id(sq_iterator))

while True:
  try:
    item = next(sq)
    print(f"Value: {item}")
  except StopIteration:
    break

# We see that the prints are the same as when python runs it through a loop.

__iter__ called
140335742015312 140335742015312
__next__ called
Value: 0
__next__ called
Value: 1
__next__ called
Value: 4
__next__ called
Value: 9
__next__ called
Value: 16
__next__ called


We still need to instantiate again the iterator to loop again, this however is not useful since we create a whole new object with the same data just to iterate over it.

We can approach the problem by splitting the collection (iterable) and the iterator.
this means that:
- The collection is **Iterable**.
  - Gets created once.
- The iteror is created every time we need to start a fresh iteration.

to accomplish this we will have:
- The iterable
  - Object that implements the `__iter__` and returns an iterator (in general, a new instance of the class)
- The iterator
  - `__iter__`, returns itself, not a new instance
  - `__next__`, returns the next element

So iterators are themselves iterables but they become exhausted when they are iterated over.

In [12]:
# First the collection
class Cities:
  def __init__(self):
    self._cities = ['Medellin', 'Cordoba', 'Cali', 'London', 'Montreal']
    self._index = 0

  def __len__(self):
    return len(self._cities)

  def __iter__(self):
    """
    this iter looks for an iterator, and finds one by instantiating a iterator
    with our object(self)
    """
    print('Cities __iter__ called')
    return CitiesIterator(self)

class CitiesIterator:
  def __init__(self, cities_obj):
    self._cities_obj = cities_obj
    self._index = 0

  def __iter__(self):
    """
    This iter looks for an iterator and finds one in self, meaning it is an
    iterator.
    """
    print(" CitiesIterator __iter__ called")
    return self

  def __next__(self):
    print("  CitiesIterator __next__ called")
    if self._index >= len(self._cities_obj):
      raise StopIteration
    else:
      item = self._cities_obj._cities[self._index]
      self._index += 1
      return item

# Now we can have a Cities class that loops by instantiating a CitiesIterator.
c1 = Cities()
for i in range(3):
  print(f"Loop {i} for the same class: {type(c1).__name__}")
  for j, city in enumerate(c1):
    print(f"\tcity:{city} in iteration: {j}")

Loop 0 for the same class: Cities
Cities __iter__ called
  CitiesIterator __next__ called
	city:Medellin in iteration: 0
  CitiesIterator __next__ called
	city:Cordoba in iteration: 1
  CitiesIterator __next__ called
	city:Cali in iteration: 2
  CitiesIterator __next__ called
	city:London in iteration: 3
  CitiesIterator __next__ called
	city:Montreal in iteration: 4
  CitiesIterator __next__ called
Loop 1 for the same class: Cities
Cities __iter__ called
  CitiesIterator __next__ called
	city:Medellin in iteration: 0
  CitiesIterator __next__ called
	city:Cordoba in iteration: 1
  CitiesIterator __next__ called
	city:Cali in iteration: 2
  CitiesIterator __next__ called
	city:London in iteration: 3
  CitiesIterator __next__ called
	city:Montreal in iteration: 4
  CitiesIterator __next__ called
Loop 2 for the same class: Cities
Cities __iter__ called
  CitiesIterator __next__ called
	city:Medellin in iteration: 0
  CitiesIterator __next__ called
	city:Cordoba in iteration: 1
  CitiesIt

In [ ]:
# Nest our CitiesIterator inside our main Cities class
class Cities:
  def __init__(self):
    self._cities = ['Medellin', 'Cordoba', 'Cali', 'London', 'Montreal']
    self._index = 0

  def __len__(self):
    return len(self._cities)

  def __iter__(self):
    """
    this iter looks for an iterator, and finds one by instantiating a iterator
    with our object(self)
    """
    print('Cities __iter__ called')
    return self.CitiesIterator(self)

  class CitiesIterator:
    def __init__(self, cities_obj):
      self._cities_obj = cities_obj
      self._index = 0

    def __iter__(self):
      """
      This iter looks for an iterator and finds one in self, meaning it is an
      iterator.
      """
      print(" CitiesIterator __iter__ called")
      return self

    def __next__(self):
      print("  CitiesIterator __next__ called")
      if self._index >= len(self._cities_obj):
        raise StopIteration
      else:
        item = self._cities_obj._cities[self._index]
        self._index += 1
        return item

# Now we can have a Cities class that loops by instantiating a CitiesIterator.
c1 = Cities()
for i in range(3):
  print(f"Loop {i} for the same class: {type(c1).__name__}")
  for j, city in enumerate(c1):
    print(f"\tcity:{city} in iteration: {j}")

We can now implement a sequence and an iterator, however, once a iteration is started the `__iter__` and `__next__` methods are prefered.

Python will:
- look for the `__iter__`, if not
- look for the `__getitem__`

In [16]:
# Sequence + Iterator protocols implemented

print("---- SEQUENCE ----")

class Cities:
  def __init__(self):
    self._cities = ['Medellin', 'Cordoba', 'Cali', 'London', 'Montreal']
    self._index = 0

  def __len__(self):
    return len(self._cities)

  def __iter__(self):
    """
    this iter looks for an iterator, and finds one by instantiating a iterator
    with our object(self)
    """
    print('Cities __iter__ called')
    return self.CitiesIterator(self)

  def __getitem__(self, i):
    return self._cities[i]

  class CitiesIterator:
    def __init__(self, cities_obj):
      self._cities_obj = cities_obj
      self._index = 0

    def __iter__(self):
      """
      This iter looks for an iterator and finds one in self, meaning it is an
      iterator.
      """
      print(" CitiesIterator __iter__ called")
      return self

    def __next__(self):
      print("  CitiesIterator __next__ called")
      if self._index >= len(self._cities_obj):
        raise StopIteration
      else:
        item = self._cities_obj._cities[self._index]
        self._index += 1
        return item

# Now we can have a Cities class that loops by instantiating a CitiesIterator.
c1 = Cities()
print(f"we also have now a cities sequence c1[0] -> {c1[0]}")
for i in range(3):
  print(f"Loop {i} for the same class: {type(c1).__name__}")
  for j, city in enumerate(c1):
    print(f"\tcity:{city} in iteration: {j}")

---- SEQUENCE ----
we also have now a cities sequence c1[0] -> Medellin
Loop 0 for the same class: Cities
Cities __iter__ called
  CitiesIterator __next__ called
	city:Medellin in iteration: 0
  CitiesIterator __next__ called
	city:Cordoba in iteration: 1
  CitiesIterator __next__ called
	city:Cali in iteration: 2
  CitiesIterator __next__ called
	city:London in iteration: 3
  CitiesIterator __next__ called
	city:Montreal in iteration: 4
  CitiesIterator __next__ called
Loop 1 for the same class: Cities
Cities __iter__ called
  CitiesIterator __next__ called
	city:Medellin in iteration: 0
  CitiesIterator __next__ called
	city:Cordoba in iteration: 1
  CitiesIterator __next__ called
	city:Cali in iteration: 2
  CitiesIterator __next__ called
	city:London in iteration: 3
  CitiesIterator __next__ called
	city:Montreal in iteration: 4
  CitiesIterator __next__ called
Loop 2 for the same class: Cities
Cities __iter__ called
  CitiesIterator __next__ called
	city:Medellin in iteration: 0
 

In [17]:
# We can verify that this is how builtins are implemented
l1 = [1,2,3,4,5]
il1 = iter(l1)
print(f"we can get the iterable for l1 -> {il1}")

for i, item in enumerate(il1):
  print(f"item: {item} at position {i} in list.")
# the iterator now gets exhausted.
try:
  next(il1)
except StopIteration:
  print("l1 iterator il1 is exhausted")

we can get the iterable for l1 -> <list_iterator object at 0x7fa27602a350>
item: 1 at position 0 in list.
item: 2 at position 1 in list.
item: 3 at position 2 in list.
item: 4 at position 3 in list.
item: 5 at position 4 in list.
l1 iterator il1 is exhausted
